# Experiments varying number of seats and candidates 


In [ ]:
import os
from model import Society
from mesa import Agent, Model
import random
from mesa.space import Grid
from mesa.datacollection import DataCollector
from mesa.time import RandomActivation
import matplotlib.pyplot as plt
import math
from mesa.space import MultiGrid
from mesa.space import SingleGrid
import numpy as np
import seaborn as sns
from tqdm import tqdm
import seaborn as sns
import time
from scipy.stats import truncnorm

import glob
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
#os.chdir("C:\\Users\\\Salome\\Desktop\\Results_Parties_Seats") #navigate to the place where you need to save the results
""" 
Experiment for varying number of seats and candidates for different values of polarization in a basic model (without interactions)
"""

# Set default model parameter values

height = 50
width = 50
population_size = 2000
C=0.3
K=0.01
iterations=2
# Create arrays of candidates, seats and  polarizations to experiment with

Parties=[2,3,5,10]
Seats=np.arange(1,20,1)
Polarization=[0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]


for parties in Parties:
    for polarization in Polarization:
            for seat in Seats:
                num_candidates=parties
                num_seats=seat 

                name ='Polarization '+str(polarization)+' Parties '+str(parties)+' Seats '+str(seat)

                # Create empty lists to save model outputs

                Happiness_FP=[] ;            Happiness_Scores=[];            Happiness_Ranking=[]

                Moderation_FP=[];            Moderation_Scores=[];            Moderation_Ranking=[]

                Parliament_FP=[];            Parliament_Scores=[];            Parliament_Ranking=[]

                # For each combination, run the model 100 times and save the model outputs
                for i in range(iterations):

                    # run the model
                    model = Society(population_size,num_candidates,num_seats,polarization,C,K)
                    model.run_model(step_count=1)
                    data = model.datacollector.get_model_vars_dataframe()       

                    Happiness_FP.append(model.happiness_fp)
                    Happiness_Scores.append(model.happiness_scores)
                    Happiness_Ranking.append(model.happiness_ranking)

                    Moderation_FP.append(model.moderation_FP)
                    Moderation_Scores.append(model.moderation_Scores)
                    Moderation_Ranking.append(model.moderation_Ranking)

                    Parliament_FP.append(model.parliament_fp)
                    Parliament_Scores.append(model.parliament_scores)
                    Parliament_Ranking.append(model.parliament_ranking)

                    np.savetxt(name+' Happiness_FP',np.array(Happiness_FP) )
                    np.savetxt(name+' Happiness_Scores',Happiness_Scores )
                    np.savetxt(name+' Happiness_Ranking',Happiness_Ranking )

                    np.savetxt(name+' Moderation_FP',Moderation_FP )
                    np.savetxt(name+' Moderation_Scores',Moderation_Scores )
                    np.savetxt(name+' Moderation_Ranking',Moderation_Ranking )

                    np.savetxt(name+' Parliament_FP',Parliament_FP )
                    np.savetxt(name+' Parliament_Scores',Parliament_Scores )
                    np.savetxt(name+' Parliament_Ranking',Parliament_Ranking )


                    

# Plotting the results

In [ ]:

'''
Observe Dissatisfaction, Moderation and Parliament distributions while varying number of seats for specific number of parties.
Each result is an average of 100 simulations.
'''
parties=2  # Define party which results are being observed:

for seat in Seats:
    '''
    Dissatisfaction distribution
    '''
    
    # Create empty lists to save model outputs and read the files.
    Happiness_FP = []
    Happiness_Scores=[]
    Happiness_Ranking=[]
    for file in glob.glob('Polarization*Parties %s Seats %s *Happiness*'%(parties,seat)): 
        if file[-1] == 'P':
            Happiness_FP.append(np.loadtxt(file))
        if file[-1] == 's':
            Happiness_Scores.append(np.loadtxt(file))
        if file[-1] == 'g':
            Happiness_Ranking.append(np.loadtxt(file))

    Averaged_Happiness_FP=[]  # for all polarizations
    Averaged_Happiness_Scores=[]
    Averaged_Happiness_Ranking=[]  
    
    # Average of 100 simulations
    for i in range(len(Polarization)):        
        Averaged_Happiness_FP.append(np.average(Happiness_FP[i], axis=0))
        Averaged_Happiness_Scores.append(np.average(Happiness_Scores[i], axis=0))
        Averaged_Happiness_Ranking.append(np.average(Happiness_Ranking[i], axis=0))

    
    # plot boxplots for dissatisfaction distribution for different voting systems
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (16,4), sharey=True)
    fig.suptitle('Dissatisfaction distribution (%s parties, %s seats)' %(parties,seat))

    labels = ['0%','10%', '20%', '30%','40%','50%', '60%', '70%' ,'80%','90%','100%']

    ax1.set_title('First Past the Post')
    ax1.set_xticklabels(labels)
    ax1.set_ylabel("Mean dissatisfaction")

    ax2.set_title('Scored Voting')
    ax2.set_xticklabels(labels)
    ax2.set_xlabel("Polarization")

    ax3.set_title('Single Transferable Vote')
    ax3.set_xticklabels(labels)

    ax1.boxplot(Averaged_Happiness_FP)
    ax2.boxplot(Averaged_Happiness_Scores)
    ax3.boxplot(Averaged_Happiness_Ranking)

    fig.tight_layout()
    fig.subplots_adjust(top = 0.8)
    plt.savefig('Dissatisfaction distribution (%s parties, %s seats)'%(parties,seat))

    plt.show()
    
    '''
    Parliament distribution
    '''
    
    Parliament_FP = []
    Parliament_Scores=[]
    Parliament_Ranking=[]

    for file in glob.glob('Polarization*Parties %s Seats %s *Parliament*'%(parties,seat)):  
        if file[-1] == 'P':
            Parliament_FP.append(np.loadtxt(file))
        if file[-1] == 's':
            Parliament_Scores.append(np.loadtxt(file))
        if file[-1] == 'g':
            Parliament_Ranking.append(np.loadtxt(file))

    Averaged_Parliament_FP=[]  
    Averaged_Parliament_Scores=[]
    Averaged_Parliament_Ranking=[]  

    for i in range(len(Polarization)):
        Averaged_Parliament_FP.append(np.average(Parliament_FP[i], axis=0))
        Averaged_Parliament_Scores.append(np.average(Parliament_Scores[i], axis=0))
        Averaged_Parliament_Ranking.append(np.average(Parliament_Ranking[i], axis=0))
    
    
    width=0.5
    ind=np.arange(1,parties+1,1)
    polar_perc=['0%','50%','100%']   # Create subplots of parliament's distribution for specific polarizations of society
    i=0
    for k in [0,5,10]:
        plt.subplot(3,1,1)
        plt.title('Parliament distribution - %s  Polarization \n (%s parties, %s seats)'%(polar_perc[i],parties,seat))
        plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=1, hspace=None)
        plt.bar(ind,Averaged_Parliament_FP[k], width, label='FPP',color='red')
        plt.xticks(ind)
        plt.legend(loc='center right')

        plt.subplot(3,1,2)
        plt.bar(ind, Averaged_Parliament_Scores[k], width, label='SV',color="darkblue")
        plt.xticks(ind)
        plt.legend(loc='center right')
        plt.ylabel('Proportion')
        plt.subplot(3,1,3)

        plt.bar(ind, Averaged_Parliament_Ranking[k], width, label='STV',color="green")
        plt.xlabel('Parties')
        plt.xticks(ind)
        plt.legend(loc='center right')
        plt.tight_layout()
        plt.savefig('Parliament distribution - %s  Polarization (%s parties, %s seats)'%(polar_perc[i],parties,seat))
        i+=1
        plt.show()

    '''
    Moderation distribution
    '''

    Moderation_FP = []
    Moderation_Scores=[]
    Moderation_Ranking=[]

    for file in glob.glob('Polarization*Parties %s Seats %s *Moderation*'%(parties,seat)):
        if file[-1] == 'P':
            Moderation_FP.append(np.loadtxt(file))
        if file[-1] == 's':
            Moderation_Scores.append(np.loadtxt(file))        
        if file[-1] == 'g':
            Moderation_Ranking.append(np.loadtxt(file))

    Averaged_Moderation_FP=np.average(Moderation_FP,axis=1)
    Averaged_Moderation_Scores=np.average(Moderation_Scores,axis=1)
    Averaged_Moderation_Ranking=np.average(Moderation_Ranking,axis=1)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (16,4), sharey=True)

    fig.suptitle('Moderation distribution (%s parties, %s seats)' %(parties,seat))
    labels = ['0%', '10%', '20%', '30%','40%','50%', '60%', '70%', '80%','90%','100%']
    ax1.set_title('First Past the Post')
    ax1.set_xticklabels(labels)
    ax1.set_ylabel("Moderation")
    ax1.grid()
    ax2.set_title('Scored Voting')
    ax2.set_xticklabels(labels)
    ax2.set_xlabel("Polarization")
    ax2.grid()
    ax3.set_title('Single Transferable Vote')
    ax3.set_xticklabels(labels)
    ax3.grid()
    ax1.boxplot([Moderation_FP[0],Moderation_FP[1],Moderation_FP[2],Moderation_FP[3],Moderation_FP[4],Moderation_FP[5],Moderation_FP[6],Moderation_FP[7],Moderation_FP[8],Moderation_FP[9],Moderation_FP[10]])
    ax2.boxplot([Moderation_Scores[0],Moderation_Scores[1],Moderation_Scores[2],Moderation_Scores[3],Moderation_Scores[4],Moderation_Scores[5],Moderation_Scores[6],Moderation_Scores[7],Moderation_Scores[8],Moderation_Scores[9],Moderation_Scores[10]])
    ax3.boxplot([Moderation_Ranking[0],Moderation_Ranking[1],Moderation_Ranking[2],Moderation_Ranking[3],Moderation_Ranking[4],Moderation_Ranking[5],Moderation_Ranking[6],Moderation_Ranking[7],Moderation_Ranking[8],Moderation_Ranking[9],Moderation_Ranking[10]])

    fig.tight_layout()
    fig.subplots_adjust(top = 0.8)
    plt.savefig('Moderation distribution (%s parties, %s seats)'%(parties,seat))

